In [ ]:
# 관련 모듈 호출
from urllib.parse import urlparse
import hashlib
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify

In [ ]:
class Blockchain(object):
    def __init__(self):
        # __init__ 블록체인 객체를 생성한다.
        # 객체의 구성 요소로는 블록들이 저장되는 체인과 블록 내에 저장될 거래 내역 리스트 current_transaction,
        # 그리고 블록체인을 운영하는 노드들의 정보인nodes,
        # 마지막으로 블록체인 첫 생성 시 자동으로 첫 블록(genesis block)을 생성하는 코드(new_block)로 구성된다.
        self.chain = []
        self.current_transaction = []
        self.nodes = set()
        self.new_block(previous_hash=1, proof=100)

    # 해시암호화 함수
    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    #마지막 블록 호출 함수
    @property
    def last_block(self):
        return self.chain[-1]
    
    #블록 검증 함수
    @staticmethod
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == '0000'
    
    #작업 증명(pow) 함수
    def pow(self, last_proof):
        proof = random.randint(-1000000, 1000000)
        while self.valid_proof(last_proof, proof) is False:
            proof = random.randint(-1000000, 1000000)
        return proof
    
    #거래 내역 추가 함수
    def new_transaction(self, sender, recipient, amount):
        self.current_transaction.append(
            {
                'sender' : sender, #송신자
                'recipient' : recipient, #수신자
                'amount' : amount, #금액
                'timestamp':time()
            }
        )
        return self.last_block['index'] + 1
    
    #신규 블록 생성 함수
    def new_block(self, proof, previous_hash = None):
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(), #timestamp from 1970
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)
        return block
    
    #블록 검증 함수
    def valid_chain(self, chain):
        last_block = chain[0]
        current_index = 1


        while current_index < len(chain):
            block = chain[current_index]
            print('%s' % last_block)
            print('%s' % block)
            print("\n-----------\n")
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
        return True

In [ ]:
# 노드 등록
def register_node(self, address):
    parsed_url = urlparse(address)
    self.nodes.add(parsed_url.netloc)

In [ ]:
# 노드의 블록 유효성 검증
def resolve_conflicts(self):
    neighbours = self.nodes # 구동되는 노드들을 저장
    new_block = None


    max_length = len(self.chain) # 내 블록의 길이 저장
    for node in neighbours:
        node_url = "http://" + str(node.replace("0.0.0.0", "localhost")) + '/chain' # url을 받아서 request 통해 체인 정보저장
        response = requests.get(node_url)
        if response.status_code == 200: # 웹페이지와 정상적으로 교류가 되면 그 정보 저장
            length = response.json()['length']
            chain = response.json()['chain']
            ## 다른 노드의 길이(length)가 내 노드의 길이(max_length)보다 길고 and 내 체인이 유효한 경우

            if length > max_length and self.valid_chain(chain): # 긴 체인을 비교 -> 제일 긴 블록이 인정된다.
                max_length = length ## 기존 노드의 정보보다 받은 정보가 최신이다. 전송받은 블록 정보를 new_block에 넣는다.
                new_block = chain ## 다른 노드의 길이(length)가 내 노드의 길이(max_length)보다 짧거나 내 체인이 유효하지 않은 경우
            else:
                1==1 # 별도 작업 불필요
            if new_block != None:
                self.chain = new_block # 기존 블록 정보가 잘못된 것을 인정하고 검증된 블록 정보로 바꾼다.
            return True
        return False

In [ ]:
# 노드 기본값 설정
blockchain = Blockchain()
my_ip = 'localhost'
my_port = '5002' # or '5000', '5001'
node_identifier = 'node_'+ my_port
mine_owner = 'master'
mine_profit = 0.1


In [ ]:
app = Flask(__name__)


#블록 정보 호출 함수
@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain,
        'length' : len(blockchain.chain),
    }
    return jsonify(response), 200


In [ ]:
# 노드 연결을 위해 추가되는 함수들
@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json() #json 형태로 보내면 노드가 저장됨
    print("register nodes !!! : ", values)
    registering_node = values.get('nodes')
    if registering_node == None: #요청된 node 값이 없다면!
        return "Error: Please supply a valid list of nodes", 400
    
    
    ## 요청 받은 노드가 이미 등록된 노드와 중복인지 검사
    ## 중복인 경우
    if registering_node.split("//")[1] in blockchain.nodes:
        print("Node already registered") #이미 등록된 노드입니다.
        response = {
            'message' : 'Already Registered Node',
            'total_nodes' : list(blockchain.nodes),
        }
    
    ## 중복이 아니라면
    else:
        # 내 노드 리스트에 추가
        blockchain.register_node(registering_node)

        ## 이후 해당 노드에 내 정보 등록하기
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        data = {
            "nodes": 'http://' + my_ip + ":" + my_port
        }
        print("MY NODE INFO ", 'http://' + my_ip + ":" + my_port)
        requests.post( registering_node + "/nodes/register", headers=headers, data=json.dumps(data))


        # 이후 주변 노드들에도 새로운 노드가 등장함을 전파
        for add_node in blockchain.nodes:
            if add_node != registering_node.split("/")[1]:
                print('add_node : ', add_node)
                ## 노드 등록하기
                headers = {'Content-Type' : 'application/json; charset=utf-8'}
                data = {
                    "nodes": registering_node
                }
                requests.post('http://' + add_node   +"/nodes/register", headers=headers, data=json.dumps(data))

                response = {
                    'message' : 'New nodes have been added',
                    'total_nodes' : list(blockchain.nodes),
                }
        return jsonify(response), 201



In [ ]:
#신규 거래 추가 함수
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount']

    if not all(k in values for k in required):
        return 'missing values', 400
    

    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message' : 'Transaction will be added to Block {%s}' % index}
    ####node.ipynb 에서 노드 연결을 위해 추가되는 부분######
    ## 본 노드에 받은 거래 내역 정보를 다른 노드들에 다 같이 업데이트 해준다.

    
    if 'type' not in values: ## 신규로 추가된 경우 type이라는 정보가 포함되어 없다. 해당 내용은 전파가 필요하다고 느낌
        for node in blockchain.nodes: #nodes에 저장된 모든 노드에 정보를 전달한다.
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender" : values['sender'],
                "recipient" : values['recipient'],
                "amount" : values['amount'],
                "type" : "sharing" #전파이기에 sharing이라는 type이 꼭 필요하다.
            }
            requests.post("http://" +node   +"/transactions/new", headers=headers, data=json.dumps(data))
            print("share transaction to >>   ","http://" + node )
    return jsonify(response), 201


In [ ]:
#채굴 함수
@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")
    last_block = blockchain.last_block
    last_proof = last_block['nonce']
    proof = blockchain.pow(last_proof)
    blockchain.new_transaction(
        sender=mine_owner,
        recipient=node_identifier,
        amount=mine_profit #coinbase transaction
    )


    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")



    ##### 노드 연결을 위해 추가되는 부분
    for node in blockchain.nodes: # nodes에 연결된 모든 노드에 작업 증명(PoW)이 완료되었음을 전파한다.
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        data = {
            "miner_node" : 'http://' + my_ip + ":" + my_port,
            "new_nonce" : blockchain.last_block['nonce']
        }


        alarm_res = requests.get("http://" + node + "/nodes/resolve", headers=headers, data = json.dumps(data) )

        if "ERROR" not in alarm_res.text : # 전파 받은 노드의 응답에 ERROR라는 이야기가 없으면(나의 PoW가 인정받으면)
            #정상 response
            response = {
            'message' : 'new block found',
            'index' : block['index'],
            'transactions' : block['transactions'],
            'nonce' : block['nonce'],
            'previous_hash' : block['previous_hash']
            }
        else: # 전파받은 노드의 응답에 이상이 있음을 알린다면?
            ## 내 PoW가 이상이 있을 수 있기에 다시 PoW 진행!
            block = blockchain.new_block(proof, previous_hash)


    return jsonify(response), 200



In [ ]:
# 타 노드에서 블록 생성 내용을 전파하였을 때 검증 작업을 진행한다.
@app.route('/nodes/resolve', methods=['GET'])
def resolve():
    requester_node_info = request.get_json()
    required = ['miner_node'] # 해당 데이터가 존재해야 함
    # 데이터가 없으면 에러를 띄움
    if not all (k in requester_node_info for k in required):
        return 'missing values', 400
    

    ## 그전에 우선 previous에서 바귄 것이 있는지 점검하자 !!
    my_previous_hash = blockchain.last_block['previous_hash']
    my_previous_hash


    last_proof = blockchain.last_block['nonce']


    headers = {'Content-Type' : 'application/json;' 'charset=utf-8'}
    miner_chain_info = requests.get(requester_node_info['miner_node'] + "/chain", headers=headers)


    ## 초기 블록은 과거 이력 변조 내역을 확인할 필요가 없다.

    print("다른노드에서 요청이 온 블록, 검증 시작")
    new_block_previous_hash = json.loads(miner_chain_info.text)['chain'][-2]['previous_hash']
    # 내 노드의 전 해시와 새로 만든 노드의 전 해시가 같을 때 !!! -> 정상
    if my_previous_hash == new_block_previous_hash and \
        hashlib.sha256(str(last_proof + int(requester_node_info['new_nonce'])).encode()).hexdigest()[:4] == "0000" :
        # 정말 PoW의 조건을 만족시켰을까? 검증하기
        print("다른노드에서 요청이 온 블록, 검증결과 정상!!!!!!!")

        replaced = blockchain.resolve_conflicts() # 결과값 : True False / True면 내 블록의 길이가 짧아 대체되어야 한다.


        # 체인 변경 알림 메시지
        if replaced == True:
            ## 내 체인이 짧아서 대체되어야 함
            print("REPLACED length : ",len(blockchain.chain))
            response = {
                'message' : 'Our chain was replaced -> ' + my_ip + ":" + my_port,
                'new_chain' : blockchain.chain
            }
        else:
            ## 내 체인이 제일 길어서 권위가 있음
            response = {
                'message' : 'Our chain is authoritative',
                'chain' : blockchain.chain
            }
        # 아니면 무엇인가 과거 데이터가 바뀐 것이다!!
    else:
        print("다른노드에서 요청이 온 블록, 검증결과 이상발생!!!!!!!")
        response = {
            'message' : "Our chain is authoritative -> " + my_ip + ":"+my_port,
            'chain' : blockchain.chain
        }
    return jsonify(response), 200

In [ ]:

#노드의 시작
if __name__=='__main__':
    app.run(host=my_ip, port=my_port)